# import

In [1]:
import os
import re
import pandas as pd
from utils import *

# 0. Load data

In [2]:
# author code
x_tr_df = pd.read_csv('train_folder/x_train_ids.csv', index_col=0)
tr_dir = "train_folder/txt_files/"
tr_files = x_tr_df['filename'].values


tr_texts = []
for tr_file in tr_files:
    f = open(os.path.join(tr_dir, tr_file), "r")
    ff = f.read()
    ff = clean1(ff)
    tr_texts.append(ff)
# tr_texts has all the texts in it

In [4]:
#y_train data
y_tr_df = pd.read_csv('train_folder/Y_train_predilex.csv', index_col=0)

In [5]:
y_tr_df.head()

,sexe,date_accident,date_consolidation
ID,,,
0,homme,1991-04-09,n.c.
1,homme,2005-06-10,2010-01-19
2,femme,1997-09-26,n.c.
3,femme,1982-08-07,1982-11-07
4,homme,1996-11-26,n.c.


# 1. Find all possible patterns for dates

In [5]:
# all index of the year of the accident 
all_dates = []
for i in range(len(tr_texts)):
    if y_tr_df['date_consolidation'][i] != 'n.c.':
        ff = [m.start() for m in re.finditer(y_tr_df['date_consolidation'][i][:4], tr_texts[i])]
    else:
        ff = []
    all_dates.append(ff)

In [6]:
# check if there is the day digit near the year digit
sentences = []
for ind in range(len(tr_texts)):
    sentence = []
    window = 20
    for date in all_dates[ind]:
        left = tr_texts[ind][date - window : date]
        right = tr_texts[ind][date + 4 : date + window]
        left_num = re.findall(r'\d', left)
        right_num = re.findall(r'\d', right)
        if len(left_num) == 0 & len(right_num) == 0:
            pass
        else:
            text = tr_texts[ind][date - window : date + window]
            sentence.append(text)
    sentences.append(sentence)

In [6]:
# find all pattern of dates
pattern1 = r"\d{1,} \w{3,9} \d{4}"
pattern2 = r"1° \w{3,9} \d{4}"
pattern3 = r"1er \w{3,9} \d{4}"
pattern4 = r"\d{1,}\.\d{2}\.\d{4}"
pattern5 = r"\d{1,}/\d{2}/\d{4}"
pattern6 = r"\d{1,} / \d{2} / \d{4}"
pattern7 = r"\d{1,}/ \d{2}/ \d{4}"
pattern8 = r"\d{1,}\. \d{2}\. \d{4}"


pattern1_bis = r"(\d{1,}) (\w{3,9}) (\d{4})"
pattern2_bis = r"(1°) (\w{3,9}) (\d{4})"
pattern3_bis = r"(1er) (\w{3,9}) (\d{4})"
pattern4_bis = r"(\d{1,})\.(\d{2})\.(\d{4})"
pattern5_bis = r"(\d{1,})/(\d{2})/(\d{4})"
pattern6_bis = r"(\d{1,}) / (\d{2}) / (\d{4})"
pattern7_bis = r"(\d{1,})/ (\d{2})/ (\d{4})"
pattern8_bis = r"(\d{1,})\. (\d{2})\. (\d{4})"

patterns = [pattern1, pattern2, pattern3, pattern4, pattern5, pattern6, pattern7, pattern8] 
patterns_bis = [pattern1_bis, pattern2_bis, pattern3_bis, pattern4_bis, pattern5_bis, pattern6_bis, pattern7_bis, pattern8_bis] 

In [8]:
# check the number of sentences that doesnt get detected using preious patterns
dates = []
fails = []
for i in range(len(sentences)):
    date = []
    index = []
    for sentence in sentences[i]:
        l = []
        for pattern in patterns:
            l.extend(re.findall(pattern, sentence))
        if len(l) == 0:
            fails.append(sentence)
        date.extend(l)
    dates.append(date)

In [9]:
len(fails)

539

In [18]:
fails

['9 cour d’appel d’agen audience publique ',
 '00908 republique francaise au nom du peu',
 ' 47910 agen cedex 9 n’ayant pas constitu',
 ' ordonnance 99-xxxx 1999-04-21 art. 1 ',
 '2 : 30. 321,00 € en 2003 : 48. 778,00 € ',
 'rance au 1o juillet 2003 il convient de ',
 'mpter du 1o juillet 2003 jusqu’au 27 aoû',
 'de juin 2000 à août 2002 (7. 447 € 66 c.',
 '/1960 republique francaise au nom du peu',
 '; 3o) à chacun des enfants jean pierre, ',
 'mai 2006 ; vu l’ordonnance de clôture en',
 'e 11 années en se fondant sur les conclu',
 '03 + 11.115,01 € en 2004) correspondant ',
 '01 et le 1o juillet 2004 des indemnités ',
 'ral sous le no 05 / 2003. appelante mada',
 '000 au mois d’avril 2001 ne permet pas à',
 'il 1998 (et non pas 1997 comme indiqué p',
 '41 €) et l’exercice 2006 (181. 527 €) qu',
 'janvier 2008 et mai 2000 ; il convient d',
 ' 2009 n° de rg: 08/02006 republique fran',
 'no/2009 rôle no 08/02006 fonds de garant',
 ' juin 2010. une ordonnance de jonction a',
 ' étant de 

# 2. Creating Training Set

In [40]:
import nltk
from tqdm import tqdm
import numpy as np
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer, SnowballStemmer
nltk.download('wordnet')
nltk.download('stopwords')

[nltk_data] Downloading package wordnet to /home/fei/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to /home/fei/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [41]:
stemmer = SnowballStemmer('french')
lemmatizer = WordNetLemmatizer()
stop_words = list(stopwords.words('french'))

In [42]:
months_str_to_int = dict(janvier='01', fevrier='02', mars='03', avril='04', mai='05', juin='06',
                         juillet='07', aout='08', septembre='09', octobre='10', novembre='11',
                         decembre='12')

def format_date(day:str, month:str):
    if len(day) == 1:
        day = '0' + day
    if not month.isnumeric():
        month = months_str_to_int[month]
    elif len(month) == 1:
        month = '0' + month
    return day, month

In [60]:
x_train = []
y_train = []
for i in tqdm(range(len(tr_texts))):
    if y_tr_df.iloc[i]['date_consolidation'] != 'n.c.':
        text = tr_texts[i]
        index = []
        dates = []
        for k, pattern in enumerate(patterns):
            index.extend(re.finditer(pattern, text))
            dates.extend(re.findall(patterns_bis[k], text))

        window = 50
        for k, ind in enumerate(index):
            if dates[k]:
                day, month, year = dates[k][0], dates[k][1], dates[k][2]
                month = month.lower().replace('é', 'e').replace('û', 'u')
                if month in months_str_to_int:
                    day, month = format_date(day, month)
                    date = f"{year}-{month}-{day}"
                    chunk = text[ind.start() - window: ind.end() + window]
                    chunk = clean2(chunk, lemmatizer, stemmer, stop_words = stop_words)
                    x_train.append(chunk)

                    if date == y_tr_df.iloc[i]['date_consolidation']: y_train.append(1)
                    else: y_train.append(0)

100%|██████████| 770/770 [00:16<00:00, 47.81it/s]


# 3. Training Model

In [61]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import KFold
from sklearn.linear_model import SGDClassifier
from xgboost import XGBClassifier
from sklearn.model_selection import cross_validate, GridSearchCV, RandomizedSearchCV

In [62]:
TF_IDF = TfidfVectorizer(ngram_range=(1, 2), max_features=100, norm='l2', sublinear_tf=True)
x_train = TF_IDF.fit_transform(x_train).toarray()

In [63]:
def perform_grid_search(cls, param_grid):
    """ perform a grid search on a classifier using 3-fold cross validation and the passed prameters """
    grid_search = GridSearchCV(estimator=cls, 
                               param_grid=param_grid,
                               scoring='accuracy',
                               cv=3,
                               verbose=1)
    grid_search.fit(x_train, y_train)
    print(f"The best hyperparameters from Grid Seach are: {grid_search.best_params_}")
    return grid_search.best_estimator_

In [64]:
num_neg = len([i for i in y_train if i == 0])
num_pos = len([i for i in y_train if i == 1])

In [65]:
num_pos

803

In [49]:
c = 0
for i in tqdm(range(len(tr_texts))):
    if y_tr_df.iloc[i]['date_consolidation'] != 'n.c.':
        c += 1
c 

100%|██████████| 770/770 [00:00<00:00, 10528.18it/s]


551

In [66]:
random_state = 42

param_grid = {
    'max_depth': [3, 5],
    'learning_rate': [.01, .1],
    'subsample': [.7, .9, 1],
    'colsample_bytree': [.7, .9, 1],
}
classifier = XGBClassifier(random_state=random_state, scale_pos_weight = num_pos/num_neg)
classifier = perform_grid_search(classifier, param_grid)

Fitting 3 folds for each of 36 candidates, totalling 108 fits


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 108 out of 108 | elapsed:   44.0s finished


The best hyperparameters from Grid Seach are: {'colsample_bytree': 1, 'learning_rate': 0.1, 'max_depth': 5, 'subsample': 1}


In [67]:
def apply_cv(cls, cv=3):
    cv_results = cross_validate(cls, x_train, y_train, scoring='accuracy', return_train_score=True, cv=cv)
    print(f"CV Accuracy: {round(cv_results['train_score'].mean() * 100, 2)}%")

In [68]:
apply_cv(classifier, 10)

CV Accuracy: 96.01%


In [69]:
classifier.fit(x_train, y_train)

XGBClassifier(base_score=0.5, booster=None, colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0, gpu_id=-1,
              importance_type='gain', interaction_constraints=None,
              learning_rate=0.1, max_delta_step=0, max_depth=5,
              min_child_weight=1, missing=nan, monotone_constraints=None,
              n_estimators=100, n_jobs=0, num_parallel_tree=1,
              objective='binary:logistic', random_state=42, reg_alpha=0,
              reg_lambda=1, scale_pos_weight=0.04294576960102685, subsample=1,
              tree_method=None, validate_parameters=False, verbosity=None)

# 4. Predict accident date in a text

In [80]:
candidates = []
sentence_dates = []
true_dates = []
for i in tqdm(range(len(tr_texts))):
    if y_tr_df.iloc[i]['date_consolidation'] != 'n.c.':
        text = tr_texts[i]
        index = []
        dates = []
        for k, pattern in enumerate(patterns):
            index.extend(re.finditer(pattern, text))
            dates.extend(re.findall(patterns_bis[k], text))

        window = 50
        data = []
        s_d = []
        for k, ind in enumerate(index):
            if dates[k]:
                day, month, year = dates[k][0], dates[k][1], dates[k][2]
                month = month.lower().replace('é', 'e').replace('û', 'u')
                if month in months_str_to_int:
                    day, month = format_date(day, month)
                    date = f"{year}-{month}-{day}"
                    chunk = text[ind.start() - window: ind.end() + window]
                    chunk = clean2(chunk, lemmatizer, stemmer, stop_words = stop_words)
                    data.append(chunk)
                    s_d.append(date)

        candidates.append(data)
        sentence_dates.append(s_d)
        true_dates.append(y_tr_df.iloc[i]['date_consolidation'])

100%|██████████| 770/770 [00:13<00:00, 55.42it/s]


In [81]:
for i in range(len(candidates)):
    candidate = TF_IDF.transform(candidates[i]).toarray()
    candidates[i] = candidate

In [82]:
preds = []
L = []
for i, candidate in enumerate(candidates):
    out = classifier.predict_proba(candidate)[:,1]
    pred = np.argmax(out)
    score = np.max(out)
    pred_date = sentence_dates[i][pred]
    preds.append(pred_date)
    if score < 0.2: L.append((score, i, pred))

In [83]:
L

[(0.1764754, 2, 12),
 (0.023688419, 3, 8),
 (0.0007391897, 4, 19),
 (0.0030952177, 5, 15),
 (0.043281507, 6, 27),
 (0.043281507, 7, 27),
 (0.0044464245, 8, 34),
 (0.11082758, 12, 11),
 (0.14186245, 13, 29),
 (0.035179198, 16, 16),
 (0.16385642, 19, 24),
 (0.090418234, 20, 16),
 (0.0012142187, 21, 14),
 (0.0007616212, 23, 7),
 (0.0010347433, 28, 23),
 (0.0051407157, 29, 41),
 (0.017217344, 32, 33),
 (0.08614001, 33, 14),
 (0.13424389, 34, 37),
 (0.17901184, 36, 17),
 (0.14963448, 39, 19),
 (0.17777017, 41, 18),
 (0.19209768, 44, 16),
 (0.08929964, 46, 25),
 (0.102056526, 51, 14),
 (0.13061894, 52, 31),
 (0.18634194, 54, 22),
 (0.089877784, 55, 19),
 (0.16455267, 57, 26),
 (0.0019466506, 58, 14),
 (0.0005454877, 59, 2),
 (0.16455267, 60, 20),
 (0.0005454877, 65, 2),
 (0.031232381, 70, 20),
 (0.096944064, 72, 18),
 (0.18516232, 75, 21),
 (0.007994383, 76, 31),
 (0.0009111358, 84, 14),
 (0.0669052, 88, 28),
 (0.14212468, 90, 13),
 (0.035921145, 91, 16),
 (0.027630374, 93, 15),
 (0.08700733

In [74]:
accuracy = 0
for i in range(len(preds)):
    if preds[i] == true_dates[i]:
        accuracy += 1
accuracy/len(preds)

0.6333938294010889

#  3. Predict test data 

In [24]:
#x_test data
x_te_df = pd.read_csv('test_folder/x_test_ids.csv', index_col=0)
te_dir = "test_folder/txt_files/"
te_files = x_te_df['filename'].values
te_texts = []
for te_file in te_files:
    f = open(os.path.join(te_dir, te_file), "r")
    ff = f.read()
    ff = clean1(ff)
    te_texts.append(ff)

In [32]:
candidates = []
sentence_dates = []
for i in tqdm(range(len(te_texts))):
    text = te_texts[i]
    index = []
    dates = []
    for k, pattern in enumerate(patterns):
        index.extend(re.finditer(pattern, text))
        dates.extend(re.findall(patterns_bis[k], text))

    window = 50
    data = []
    s_d = []
    for k, ind in enumerate(index):
        if dates[k]:
            day, month, year = dates[k][0], dates[k][1], dates[k][2]
            month = month.lower().replace('é', 'e').replace('û', 'u')
            if month in months_str_to_int:
                day, month = format_date(day, month)
                date = f"{year}-{month}-{day}"
                chunk = text[ind.start() - window: ind.end() + window]
                chunk = clean2(chunk, lemmatizer, stemmer, stop_words = stop_words)
                data.append(chunk)
                s_d.append(date)

    candidates.append(data)
    sentence_dates.append(s_d)

100%|██████████| 257/257 [00:05<00:00, 44.68it/s]


In [33]:
for i in range(len(candidates)):
    candidate = TF_IDF.transform(candidates[i]).toarray()
    candidates[i] = candidate

In [34]:
preds = []
for i, candidate in enumerate(candidates):
    pred = np.argmax(classifier.predict_proba(candidate)[:,1])
    pred_date = sentence_dates[i][pred]
    preds.append(pred_date)

In [36]:
np.save('results/consolidation.npy', preds)